# Antenna Clustering Optimization - Comparison

Questo notebook confronta l'algoritmo originale Monte Carlo con la versione ottimizzata.

**Ottimizzazioni implementate:**
1. Greedy initialization (iterazioni 1-10)
2. Local search refinement
3. Adaptive probability sampling (iterazioni 51+)
4. NumPy vectorization nel kernel computation

## 1. Setup e Import

In [ ]:
# Installa dipendenze se su Colab
import sys
if 'google.colab' in sys.modules:
    print("Running on Colab - installing dependencies...")
    !pip install numpy scipy matplotlib -q
    print("Done!")

In [ ]:
import numpy as np
import time
from dataclasses import dataclass, field
from typing import List, Tuple, Dict, Optional
from scipy.interpolate import griddata
import matplotlib.pyplot as plt

print("Import completati!")

## 2. Definizione Classi di Configurazione

In [ ]:
@dataclass
class LatticeConfig:
    """Configurazione lattice array"""
    Nz: int  # Number of rows
    Ny: int  # Number of columns
    dist_z: float  # antenna distance on z axis [times lambda]
    dist_y: float  # antenna distance on y axis [times lambda]
    lattice_type: int = 1  # 1=Rectangular

@dataclass
class SystemConfig:
    """Parametri sistema"""
    freq: float  # [Hz]
    lambda_: float = field(init=False)
    beta: float = field(init=False)
    azi0: float = 0.0
    ele0: float = 0.0
    dele: float = 0.5
    dazi: float = 0.5

    def __post_init__(self):
        self.lambda_ = 3e8 / self.freq
        self.beta = 2 * np.pi / self.lambda_

@dataclass
class MaskConfig:
    """Parametri maschera SLL"""
    elem: float = 30.0
    azim: float = 60.0
    SLL_level: float = 20.0
    SLLin: float = 15.0

@dataclass
class ElementPatternConfig:
    """Configurazione pattern elemento"""
    P: int = 1
    Gel: float = 5.0
    load_file: int = 0

@dataclass
class SimulationConfig:
    """Parametri simulazione"""
    Niter: int = 1000
    Cost_thr: int = 1000

@dataclass
class ClusterConfig:
    """Configurazione cluster"""
    Cluster_type: List[np.ndarray] = field(default_factory=list)
    rotation_cluster: int = 0

    def __post_init__(self):
        if not self.Cluster_type:
            self.Cluster_type = [np.array([[0, 0], [0, 1]])]

print("Classi di configurazione definite!")

## 3. Classe AntennaArray (con ottimizzazioni kernel)

In [ ]:
class AntennaArray:
    """Classe per array di antenne con clustering"""

    def __init__(self, lattice: LatticeConfig, system: SystemConfig, 
                 mask: MaskConfig, eef_config: Optional[ElementPatternConfig] = None):
        self.lattice = lattice
        self.system = system
        self.mask = mask
        self.eef_config = eef_config or ElementPatternConfig()
        self.Nel = lattice.Nz * lattice.Ny

        self._compute_lattice_vectors()
        self._generate_lattice()
        self._generate_polar_coordinates()
        self._generate_element_pattern()
        self._generate_mask()

    def _compute_lattice_vectors(self):
        lambda_ = self.system.lambda_
        dz = self.lattice.dist_z * lambda_
        dy = self.lattice.dist_y * lambda_
        self.x1 = np.array([dy, 0.0])
        self.x2 = np.array([0.0, dz])

    def _generate_lattice(self):
        Nz = self.lattice.Nz
        Ny = self.lattice.Ny

        if Nz % 2 == 1:
            M = np.arange(-(Nz - 1) / 2, (Nz - 1) / 2 + 1)
        else:
            M = np.arange(-Nz / 2 + 1, Nz / 2 + 1)

        if Ny % 2 == 1:
            N = np.arange(-(Ny - 1) / 2, (Ny - 1) / 2 + 1)
        else:
            N = np.arange(-Ny / 2 + 1, Ny / 2 + 1)

        self.NN, self.MM = np.meshgrid(N, M)
        dz = self.x2[1]
        dy = self.x1[0]
        DELTA = max(self.x2[0], self.x1[1])

        self.Y = self.NN * dy
        self.Z = self.MM * dz
        self.Y[1::2, :] = self.Y[1::2, :] + DELTA

        self.Dz = np.max(self.Z) - np.min(self.Z)
        self.Dy = np.max(self.Y) - np.min(self.Y)
        self.Dy_total = self.Dy + self.x1[0]
        self.Dz_total = self.Dz + self.x2[1]
        self.ArrayMask = np.ones_like(self.Y)

    def _generate_polar_coordinates(self):
        beta = self.system.beta
        lambda_ = self.system.lambda_

        self.ele = np.arange(-90, 90 + self.system.dele, self.system.dele)
        self.azi = np.arange(-90, 90 + self.system.dazi, self.system.dazi)
        self.AZI, self.ELE = np.meshgrid(self.azi, self.ele)

        self.WWae = beta * np.cos(np.deg2rad(90 - self.ELE))
        self.Vvae = beta * np.sin(np.deg2rad(90 - self.ELE)) * np.sin(np.deg2rad(self.AZI))

        chi = 2
        self.Nw = int(np.floor(chi * 4 * self.Dz_total / lambda_))
        self.Nv = int(np.floor(chi * 4 * self.Dy_total / lambda_))

        ww = np.linspace(0, beta, self.Nw + 1)
        self.ww = np.concatenate([-np.flip(ww[1:]), ww])
        vv = np.linspace(0, beta, self.Nv + 1)
        self.vv = np.concatenate([-np.flip(vv[1:]), vv])

        self.WW, self.VV = np.meshgrid(self.ww, self.vv)

        WW_clipped = np.clip(self.WW / beta, -1, 1)
        self.ELEi = 90 - np.rad2deg(np.arccos(WW_clipped))

        denom = beta * np.sin(np.deg2rad(90 - self.ELEi))
        with np.errstate(divide="ignore", invalid="ignore"):
            ratio = np.clip(self.VV / denom, -1, 1)
            self.AZIi = np.real(np.rad2deg(np.arcsin(ratio)))

        self.AZIi[self.Nv, :] = 0
        self.AZIi[self.Nv, 0] = 90
        self.AZIi[self.Nv, -1] = 90

    def _generate_element_pattern(self):
        P = self.eef_config.P
        Gel = self.eef_config.Gel

        if P == 0:
            self.Fel = np.ones_like(self.ELE)
        else:
            self.Fel = (np.cos(np.deg2rad(self.ELE * 0.9)) * np.cos(np.deg2rad(self.AZI * 0.9))) ** P

        if P == 0:
            self.Fel_VW = np.ones_like(self.ELEi)
        else:
            self.Fel_VW = (np.cos(np.deg2rad(self.ELEi * 0.9)) * np.cos(np.deg2rad(self.AZIi * 0.9))) ** P

        self.RPE = 20 * np.log10(np.abs(self.Fel) + 1e-10)
        self.RPE_ele_max = np.max(self.RPE) + Gel
        self.G_boresight = self.RPE_ele_max + 10 * np.log10(self.Nel)

    def _generate_mask(self):
        ele0 = self.system.ele0
        azi0 = self.system.azi0
        elem = self.mask.elem
        azim = self.mask.azim
        SLL_level = self.mask.SLL_level
        SLLin = self.mask.SLLin

        ele_cond = np.abs(self.ELE - ele0) <= elem
        azi_cond = np.abs(self.AZI - azi0) <= azim

        self.in_fov_mask = ele_cond & azi_cond
        self.out_fov_mask = ~self.in_fov_mask

        self.Mask_EA = np.full_like(self.ELE, self.G_boresight - SLL_level, dtype=float)
        self.Mask_EA[self.in_fov_mask] = self.G_boresight - SLLin

    def index_to_position_cluster(self, Cluster: List[np.ndarray], 
                                   ElementExc: Optional[np.ndarray] = None):
        if ElementExc is None:
            ElementExc = np.ones((self.lattice.Nz, self.lattice.Ny))

        Ntrans = len(Cluster)
        max_Lsub = max(c.shape[0] for c in Cluster)

        Yc = np.full((max_Lsub, Ntrans), np.nan)
        Zc = np.full((max_Lsub, Ntrans), np.nan)
        Ac = np.full((max_Lsub, Ntrans), np.nan)

        min_NN = int(np.min(self.NN))
        min_MM = int(np.min(self.MM))

        for kk, cluster in enumerate(Cluster):
            for l1 in range(cluster.shape[0]):
                Iy = int(cluster[l1, 0] - min_NN)
                Iz = int(cluster[l1, 1] - min_MM)
                Yc[l1, kk] = self.Y[Iz, Iy]
                Zc[l1, kk] = self.Z[Iz, Iy]
                Ac[l1, kk] = ElementExc[Iz, Iy]

        return Yc, Zc, Ac

    def coefficient_evaluation(self, Zc_m: np.ndarray, Yc_m: np.ndarray, Lsub: np.ndarray):
        beta = self.system.beta
        ele0 = self.system.ele0
        azi0 = self.system.azi0

        v0 = beta * np.sin(np.deg2rad(90 - ele0)) * np.sin(np.deg2rad(azi0))
        w0 = beta * np.cos(np.deg2rad(90 - ele0))
        Phase_m = np.exp(-1j * (w0 * Zc_m + v0 * Yc_m))
        Amplit_m = 1.0 / Lsub
        c0 = Amplit_m * Phase_m
        return c0

    def kernel1_rpe(self, Lsub: np.ndarray, Ac: np.ndarray, 
                    Yc: np.ndarray, Zc: np.ndarray, c0: np.ndarray):
        Ntrans = len(Lsub)

        # OPT: Cache flattened arrays
        if not hasattr(self, '_VV_flat'):
            self._VV_flat = self.VV.flatten()
            self._WW_flat = self.WW.flatten()
            self._Fel_VW_flat = self.Fel_VW.flatten()

        VV_flat = self._VV_flat
        WW_flat = self._WW_flat
        Fel_VW_flat = self._Fel_VW_flat
        Npoints = len(VV_flat)

        # OPT: Vectorized kernel computation
        all_Y = []
        all_Z = []
        cluster_indices = []

        for kk in range(Ntrans):
            Lsub_k = int(Lsub[kk])
            for jj in range(Lsub_k):
                if not np.isnan(Yc[jj, kk]) and not np.isnan(Zc[jj, kk]):
                    all_Y.append(Yc[jj, kk])
                    all_Z.append(Zc[jj, kk])
                    cluster_indices.append(kk)

        all_Y = np.array(all_Y)
        all_Z = np.array(all_Z)
        cluster_indices = np.array(cluster_indices)

        # OPT: Compute all phases at once using broadcasting
        phases = np.exp(1j * (np.outer(VV_flat, all_Y) + np.outer(WW_flat, all_Z)))
        phases = phases * Fel_VW_flat[:, np.newaxis]

        # OPT: Sum contributions using np.add.at
        KerFF_sub = np.zeros((Npoints, Ntrans), dtype=complex)
        np.add.at(KerFF_sub.T, cluster_indices, phases.T)

        FF = KerFF_sub @ c0.T
        FF_norm = FF / (np.max(np.abs(FF)) + 1e-10)
        FF_norm_2D = FF_norm.reshape(self.VV.shape)
        FF_norm_dB = 20 * np.log10(np.abs(FF_norm_2D) + 1e-10)

        # OPT: Cache interpolation setup
        if not hasattr(self, '_interp_points'):
            self._interp_points = np.column_stack([WW_flat, VV_flat])
            self._interp_xi = np.column_stack([self.WWae.flatten(), self.Vvae.flatten()])

        values = FF_norm_dB.flatten()
        FF_I_dB_flat = griddata(self._interp_points, values, self._interp_xi, 
                                 method="linear", fill_value=-100)
        FF_I_dB = FF_I_dB_flat.reshape(self.WWae.shape)

        Nel_active = np.sum(Lsub)
        FF_I_dB = FF_I_dB + self.RPE_ele_max + 10 * np.log10(Nel_active)

        return FF_norm_dB, FF_I_dB, KerFF_sub, np.abs(FF_norm_2D) ** 2

    def compute_cost_function(self, FF_I_dB: np.ndarray) -> int:
        Constr = FF_I_dB - self.Mask_EA
        Cm = np.sum(Constr > 0)
        return int(Cm)

    def compute_sll(self, FF_I_dB: np.ndarray):
        sll_out_values = FF_I_dB[self.out_fov_mask]
        sll_out = np.max(sll_out_values) if len(sll_out_values) > 0 else -100

        sll_in_values = FF_I_dB[self.in_fov_mask]
        if len(sll_in_values) > 0:
            max_val = np.max(sll_in_values)
            sll_in = np.max(sll_in_values[sll_in_values < max_val - 0.1]) if np.any(sll_in_values < max_val - 0.1) else max_val
        else:
            sll_in = -100

        return sll_in, sll_out

    def evaluate_clustering(self, Cluster: List[np.ndarray], 
                            ElementExc: Optional[np.ndarray] = None) -> Dict:
        if ElementExc is None:
            ElementExc = np.ones((self.lattice.Nz, self.lattice.Ny))

        Yc, Zc, Ac = self.index_to_position_cluster(Cluster, ElementExc)
        Ntrans = len(Cluster)

        Lsub = np.array([c.shape[0] for c in Cluster])
        Zc_m = np.array([np.nanmean(Zc[:Lsub[k], k]) for k in range(Ntrans)])
        Yc_m = np.array([np.nanmean(Yc[:Lsub[k], k]) for k in range(Ntrans)])

        c0 = self.coefficient_evaluation(Zc_m, Yc_m, Lsub)
        FF_norm_dB, FF_I_dB, KerFF_sub, FF_norm = self.kernel1_rpe(Lsub, Ac, Yc, Zc, c0)

        Cm = self.compute_cost_function(FF_I_dB)
        sll_in, sll_out = self.compute_sll(FF_I_dB)

        max_idx = np.unravel_index(np.argmax(FF_I_dB), FF_I_dB.shape)
        theta_max = self.ele[max_idx[0]]
        phi_max = self.azi[max_idx[1]]

        Iele = np.argmin(np.abs(self.ele - self.system.ele0))
        Iazi = np.argmin(np.abs(self.azi - self.system.azi0))

        G_boresight = self.RPE_ele_max + 10 * np.log10(np.sum(Lsub))
        SL_maxpointing = G_boresight - FF_I_dB[max_idx]
        SL_theta_phi = G_boresight - FF_I_dB[Iele, Iazi]

        return {
            "Yc": Yc, "Zc": Zc, "Ac": Ac, "Yc_m": Yc_m, "Zc_m": Zc_m,
            "Lsub": Lsub, "Ntrans": Ntrans, "c0": c0,
            "FF_norm_dB": FF_norm_dB, "FF_I_dB": FF_I_dB, "KerFF_sub": KerFF_sub,
            "Cm": Cm, "sll_in": sll_in, "sll_out": sll_out,
            "theta_max": theta_max, "phi_max": phi_max,
            "SL_maxpointing": SL_maxpointing, "SL_theta_phi": SL_theta_phi,
            "G_boresight": G_boresight,
        }

print("Classe AntennaArray definita!")

## 4. Classe per generazione Subarray

In [ ]:
class FullSubarraySetGeneration:
    """Genera il set completo di subarray possibili"""

    def __init__(self, cluster_type: np.ndarray, lattice: LatticeConfig,
                 NN: np.ndarray, MM: np.ndarray, rotation_cluster: int = 0):
        self.cluster_type = np.atleast_2d(cluster_type)
        self.lattice = lattice
        self.NN = NN
        self.MM = MM
        self.rotation_cluster = rotation_cluster
        self.S, self.Nsub = self._generate()

    def _generate(self):
        B = self.cluster_type
        A = np.sum(B, axis=0)
        M = self.MM.flatten()
        N = self.NN.flatten()

        if A[0] == 0:
            step_M = B.shape[0]
            step_N = 1
        elif A[1] == 0:
            step_N = B.shape[0]
            step_M = 1
        else:
            step_M = 1
            step_N = 1

        S = []
        min_M, max_M = int(np.min(M)), int(np.max(M))
        min_N, max_N = int(np.min(N)), int(np.max(N))

        for kk in range(min_M, max_M + 1, step_M):
            for hh in range(min_N, max_N + 1, step_N):
                Bshift = B.copy()
                Bshift[:, 0] = B[:, 0] + hh
                Bshift[:, 1] = B[:, 1] + kk

                check = not np.any(
                    (Bshift[:, 0] > max_N) | (Bshift[:, 0] < min_N) |
                    (Bshift[:, 1] > max_M) | (Bshift[:, 1] < min_M)
                )

                if check:
                    S.append(Bshift)

        return S, len(S)

print("Classe FullSubarraySetGeneration definita!")

## 5. Classe Optimizer (con tutte le ottimizzazioni)

In [ ]:
class IrregularClusteringMonteCarlo:
    """Ottimizzazione clustering con approccio Monte Carlo + ottimizzazioni"""

    def __init__(self, array: AntennaArray, cluster_config: ClusterConfig,
                 sim_config: SimulationConfig):
        self.array = array
        self.cluster_config = cluster_config
        self.sim_config = sim_config

        self.S_all = []
        self.N_all = []
        self.L = []

        for bb, cluster_type in enumerate(cluster_config.Cluster_type):
            gen = FullSubarraySetGeneration(
                cluster_type, array.lattice, array.NN, array.MM,
                cluster_config.rotation_cluster
            )
            self.S_all.append(gen.S)
            self.N_all.append(gen.Nsub)
            self.L.append(cluster_type.shape[0])

        self.simulation = []
        self.all_Cm = []
        self.all_Ntrans = []
        self.all_Nel = []

        # OPT: Adaptive probability tracking
        self.total_clusters = sum(self.N_all)
        self._cluster_scores = np.ones(self.total_clusters)
        self._selection_counts = np.ones(self.total_clusters)

    def _select_random_clusters(self):
        """Selezione random originale (probabilità 50%)"""
        selected_clusters = []
        selected_rows = []

        for bb, S in enumerate(self.S_all):
            Nsub = self.N_all[bb]
            selection = np.random.randint(0, 2, size=Nsub)
            selected_rows.append(selection)
            for idx in np.where(selection == 1)[0]:
                selected_clusters.append(S[idx])

        return selected_clusters, np.concatenate(selected_rows)

    def _select_adaptive_clusters(self):
        """OPT: Selezione con probabilità adattiva"""
        selected_clusters = []
        selected_rows = []

        probs = self._cluster_scores / self._selection_counts
        probs = np.clip(probs, 0.1, 0.9)

        offset = 0
        for bb, S in enumerate(self.S_all):
            Nsub = self.N_all[bb]
            cluster_probs = probs[offset : offset + Nsub]
            selection = (np.random.random(Nsub) < cluster_probs).astype(int)
            selected_rows.append(selection)
            for idx in np.where(selection == 1)[0]:
                selected_clusters.append(S[idx])
            offset += Nsub

        return selected_clusters, np.concatenate(selected_rows)

    def _update_adaptive_scores(self, selected_rows: np.ndarray, Cm: int):
        """OPT: Aggiorna score adattivi"""
        reward = max(0, self.sim_config.Cost_thr - Cm) / self.sim_config.Cost_thr
        indices = np.where(selected_rows == 1)[0]
        self._cluster_scores[indices] += reward
        self._selection_counts[indices] += 1

    def _greedy_initialization(self, max_clusters: int = None):
        """OPT: Inizializzazione greedy"""
        if max_clusters is None:
            max_clusters = self.total_clusters // 2

        all_clusters = []
        cluster_to_flat_idx = []
        offset = 0
        for bb, S in enumerate(self.S_all):
            for idx, cluster in enumerate(S):
                all_clusters.append(cluster)
                cluster_to_flat_idx.append(offset + idx)
            offset += self.N_all[bb]

        covered_elements = set()
        selected_flat_indices = []

        available_indices = list(range(len(all_clusters)))
        np.random.shuffle(available_indices)

        for idx in available_indices:
            if len(selected_flat_indices) >= max_clusters:
                break
            cluster = all_clusters[idx]
            cluster_elements = set(tuple(pos) for pos in cluster)
            overlap = cluster_elements & covered_elements
            if len(overlap) == 0:
                selected_flat_indices.append(cluster_to_flat_idx[idx])
                covered_elements.update(cluster_elements)

        selected_rows = np.zeros(self.total_clusters, dtype=int)
        selected_rows[selected_flat_indices] = 1

        selected_clusters = []
        offset = 0
        for bb, S in enumerate(self.S_all):
            Nsub = self.N_all[bb]
            for idx in range(Nsub):
                if selected_rows[offset + idx] == 1:
                    selected_clusters.append(S[idx])
            offset += Nsub

        return selected_clusters, selected_rows

    def _rows_to_clusters(self, selected_rows: np.ndarray):
        """OPT: Converte array selezione in lista cluster"""
        clusters = []
        offset = 0
        for bb, S in enumerate(self.S_all):
            Nsub = self.N_all[bb]
            for idx in range(Nsub):
                if selected_rows[offset + idx] == 1:
                    clusters.append(S[idx])
            offset += Nsub
        return clusters

    def _local_search(self, selected_rows: np.ndarray, current_Cm: int, 
                      max_iterations: int = 50):
        """OPT: Local search per raffinamento"""
        best_rows = selected_rows.copy()
        best_Cm = current_Cm

        for _ in range(max_iterations):
            improved = False
            indices_to_try = np.random.permutation(self.total_clusters)[:min(20, self.total_clusters)]

            for idx in indices_to_try:
                candidate_rows = best_rows.copy()
                candidate_rows[idx] = 1 - candidate_rows[idx]
                clusters = self._rows_to_clusters(candidate_rows)
                if len(clusters) == 0:
                    continue

                result = self.array.evaluate_clustering(clusters)
                candidate_Cm = result["Cm"]

                if candidate_Cm < best_Cm:
                    best_rows = candidate_rows
                    best_Cm = candidate_Cm
                    improved = True
                    break

            if not improved:
                break

        return best_rows, best_Cm

    def run(self, verbose: bool = True, use_optimizations: bool = True) -> Dict:
        """Esegue ottimizzazione"""
        start_time = time.time()

        if verbose:
            print("=" * 60)
            print("IRREGULAR CLUSTERING - MONTE CARLO OPTIMIZATION")
            if use_optimizations:
                print("  [OPTIMIZED MODE: greedy init + local search + adaptive]")
            else:
                print("  [ORIGINAL MODE: random sampling only]")
            print("=" * 60)
            print(f"Array: {self.array.lattice.Nz}x{self.array.lattice.Ny} = {self.array.Nel} elementi")
            print(f"Iterazioni: {self.sim_config.Niter}")
            print("=" * 60)

        sss = 0
        best_Cm_so_far = float("inf")

        for ij_cont in range(1, self.sim_config.Niter + 1):
            # Progress bar
            if verbose and ij_cont % 10 == 0:
                pct = (ij_cont / self.sim_config.Niter) * 100
                print(f"  [Progresso: {ij_cont}/{self.sim_config.Niter} ({pct:.0f}%) | Best Cm: {best_Cm_so_far:.0f}]", end="\r")

            # Selezione cluster
            if use_optimizations and ij_cont <= 10:
                if verbose and ij_cont == 1:
                    print("  >> Fase 1: Greedy initialization (iter 1-10)")
                Cluster, selected_rows = self._greedy_initialization()
            elif use_optimizations and ij_cont == 11:
                if verbose:
                    print("\n  >> Fase 2: Random sampling (iter 11-50)")
                Cluster, selected_rows = self._select_random_clusters()
            elif use_optimizations and ij_cont == 51:
                if verbose:
                    print("\n  >> Fase 3: Adaptive sampling (iter 51+)")
                Cluster, selected_rows = self._select_adaptive_clusters()
            elif use_optimizations and ij_cont > 50:
                Cluster, selected_rows = self._select_adaptive_clusters()
            else:
                Cluster, selected_rows = self._select_random_clusters()

            if len(Cluster) == 0:
                self.all_Cm.append(float("inf"))
                self.all_Ntrans.append(0)
                self.all_Nel.append(0)
                continue

            result = self.array.evaluate_clustering(Cluster)
            Cm = result["Cm"]
            Ntrans = result["Ntrans"]
            Nel_active = int(np.sum(result["Lsub"]))

            # Local search
            if use_optimizations and Cm < self.sim_config.Cost_thr * 2:
                old_Cm = Cm
                selected_rows, Cm = self._local_search(selected_rows, Cm, max_iterations=30)
                if verbose and Cm < old_Cm:
                    print(f"\n  >> Local search: Cm {old_Cm} -> {Cm}")
                Cluster = self._rows_to_clusters(selected_rows)
                if len(Cluster) > 0:
                    result = self.array.evaluate_clustering(Cluster)
                    Cm = result["Cm"]
                    Ntrans = result["Ntrans"]
                    Nel_active = int(np.sum(result["Lsub"]))

            if use_optimizations:
                self._update_adaptive_scores(selected_rows, Cm)

            self.all_Cm.append(Cm)
            self.all_Ntrans.append(Ntrans)
            self.all_Nel.append(Nel_active)

            if Cm < best_Cm_so_far:
                best_Cm_so_far = Cm

            if Cm < self.sim_config.Cost_thr:
                sss += 1
                solution = {
                    "selected_rows": selected_rows.copy(),
                    "Cm": Cm,
                    "Ntrans": Ntrans,
                    "Nel": Nel_active,
                    "sll_in": result["sll_in"],
                    "sll_out": result["sll_out"],
                    "iteration": ij_cont,
                }
                self.simulation.append(solution)

        elapsed_time = time.time() - start_time

        if verbose:
            print("\n")
            print("=" * 60)
            print("RISULTATI")
            print("=" * 60)
            print(f"Iterazioni: {self.sim_config.Niter}")
            print(f"Soluzioni valide: {len(self.simulation)}")
            print(f"Tempo: {elapsed_time:.2f} s")

            if self.simulation:
                best_sol = min(self.simulation, key=lambda x: x["Cm"])
                print(f"\nMIGLIORE SOLUZIONE:")
                print(f"  Cm: {best_sol['Cm']}")
                print(f"  Ntrans: {best_sol['Ntrans']}")
                print(f"  Nel: {best_sol['Nel']}")
                print(f"  SLL out: {best_sol['sll_out']:.2f} dB")
                print(f"  SLL in: {best_sol['sll_in']:.2f} dB")

        return {
            "simulation": self.simulation,
            "all_Cm": self.all_Cm,
            "all_Ntrans": self.all_Ntrans,
            "all_Nel": self.all_Nel,
            "elapsed_time": elapsed_time,
            "n_valid_solutions": len(self.simulation),
        }

print("Classe IrregularClusteringMonteCarlo definita!")

## 6. Configurazione

In [ ]:
# Parametri configurabili
NITER = 200  # Numero iterazioni (aumenta per risultati migliori)
SEED = 42    # Random seed per riproducibilità

# Configurazione array
lattice = LatticeConfig(Nz=16, Ny=16, dist_z=0.6, dist_y=0.53, lattice_type=1)
system = SystemConfig(freq=29.5e9, azi0=0, ele0=0, dele=0.5, dazi=0.5)
mask = MaskConfig(elem=30, azim=60, SLL_level=20, SLLin=15)
eef = ElementPatternConfig(P=1, Gel=5, load_file=0)
cluster_config = ClusterConfig(Cluster_type=[np.array([[0, 0], [0, 1]])])
sim_config = SimulationConfig(Niter=NITER, Cost_thr=1000)

print("Configurazione:")
print(f"  Array: {lattice.Nz}x{lattice.Ny} = {lattice.Nz * lattice.Ny} elementi")
print(f"  Frequenza: {system.freq/1e9:.1f} GHz")
print(f"  Iterazioni: {NITER}")
print(f"  Cost threshold: {sim_config.Cost_thr}")

## 7. Inizializzazione Array

In [ ]:
print("Inizializzazione array antenna...")
array = AntennaArray(lattice, system, mask, eef)
print("Array inizializzato!")

## 8. Test ORIGINALE (senza ottimizzazioni)

In [ ]:
print("="*70)
print("TEST ALGORITMO ORIGINALE (use_optimizations=False)")
print("="*70)

np.random.seed(SEED)
optimizer_original = IrregularClusteringMonteCarlo(array, cluster_config, sim_config)
results_original = optimizer_original.run(verbose=True, use_optimizations=False)

## 9. Test OTTIMIZZATO

In [ ]:
print("="*70)
print("TEST ALGORITMO OTTIMIZZATO (use_optimizations=True)")
print("="*70)

np.random.seed(SEED)
optimizer_optimized = IrregularClusteringMonteCarlo(array, cluster_config, sim_config)
results_optimized = optimizer_optimized.run(verbose=True, use_optimizations=True)

## 10. Confronto Risultati

In [ ]:
print("="*70)
print("CONFRONTO RISULTATI")
print("="*70)

def get_best(results):
    if results["n_valid_solutions"] == 0:
        return None
    return min(results["simulation"], key=lambda x: x["Cm"])

best_orig = get_best(results_original)
best_opt = get_best(results_optimized)

print(f"{'Metrica':<30} {'Originale':>15} {'Ottimizzato':>15}")
print("-"*60)
print(f"{'Tempo esecuzione (s)':<30} {results_original['elapsed_time']:>15.2f} {results_optimized['elapsed_time']:>15.2f}")
print(f"{'Soluzioni valide':<30} {results_original['n_valid_solutions']:>15} {results_optimized['n_valid_solutions']:>15}")

if best_orig and best_opt:
    print(f"{'Best Cm (lower=better)':<30} {best_orig['Cm']:>15} {best_opt['Cm']:>15}")
    print(f"{'Best SLL out (dB)':<30} {best_orig['sll_out']:>15.2f} {best_opt['sll_out']:>15.2f}")
    print(f"{'Best Ntrans':<30} {best_orig['Ntrans']:>15} {best_opt['Ntrans']:>15}")
    print(f"{'Best Nel':<30} {best_orig['Nel']:>15} {best_opt['Nel']:>15}")

print("\n" + "="*70)

## 11. Visualizzazione

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Cost function evolution
axes[0].plot(results_original['all_Cm'], 'b-', alpha=0.7, label='Originale')
axes[0].plot(results_optimized['all_Cm'], 'r-', alpha=0.7, label='Ottimizzato')
axes[0].set_xlabel('Iterazione')
axes[0].set_ylabel('Cost Function (Cm)')
axes[0].set_title('Evoluzione Cost Function')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Histogram of Cm values
cm_orig = [c for c in results_original['all_Cm'] if c != float('inf') and c < 5000]
cm_opt = [c for c in results_optimized['all_Cm'] if c != float('inf') and c < 5000]
axes[1].hist(cm_orig, bins=30, alpha=0.5, label='Originale', color='blue')
axes[1].hist(cm_opt, bins=30, alpha=0.5, label='Ottimizzato', color='red')
axes[1].set_xlabel('Cost Function (Cm)')
axes[1].set_ylabel('Frequenza')
axes[1].set_title('Distribuzione Cost Function')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Number of clusters
axes[2].plot(results_original['all_Ntrans'], 'b-', alpha=0.7, label='Originale')
axes[2].plot(results_optimized['all_Ntrans'], 'r-', alpha=0.7, label='Ottimizzato')
axes[2].set_xlabel('Iterazione')
axes[2].set_ylabel('Numero Cluster')
axes[2].set_title('Numero di Cluster per Iterazione')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 12. Conclusioni

**Ottimizzazioni implementate:**
1. **Greedy initialization** (iter 1-10): Costruisce soluzioni iniziali selezionando cluster non sovrapposti
2. **Local search** (su soluzioni promettenti): Migliora flip-by-flip le soluzioni buone
3. **Adaptive sampling** (iter 51+): I cluster che appaiono in buone soluzioni hanno maggiore probabilità
4. **NumPy vectorization**: Kernel computation ottimizzato con operazioni vettoriali

Ogni riga di codice ottimizzata ha un commento `OPT:` che spiega cosa fa.